In [1]:
%autosave 0

Autosave disabled


# The Davis-Putnam Algorithm

This notebook implements the algorithm of Davis and Putnam.  Further details about this algorithm are provided in the lecture notes.

The function $\texttt{complement}(l)$ computes the complement of a literal $l$.
If $p$ is a propositional variable, we have the following: 
<ol>
    <li>$\texttt{complement}(p) = \neg p$,
    </li>
    <li>$\texttt{complement}(\neg p) = p$.
    </li>
</ol>

In [2]:
def complement(l):
    "Compute the complement of the literal L."
    if isinstance(l, str):
        return ('¬', l)
    else:
        return l[1]

The function $\texttt{extractVariable}(l)$ extracts the variable from the literal $l$.
If $p$ is a propositional variable, we have the following: 
<ol>
    <li>$\texttt{extractVariable}(p) = p$,
    </li>
    <li>$\texttt{extractVariable}(\neg p) = p$.
    </li>
</ol>

In [3]:
def extractVariable(l):
    "Extract the propositional variable from the literal L."
    if isinstance(l, str):
        return l
    else:
        return l[1]

The function $\texttt{arb}(S)$ returns an arbitrary element from the set $S$.

In [4]:
def arb(S):
    "Return some member from the set S."
    for x in S:
        return x

We select an arbitrary variable from an arbitrary clause from the set <tt>Clauses</tt> that is not a member of the set of variables in the set <tt>Forbidden</tt>.

In [5]:
def selectVariable(Clauses, Forbidden):
    Variables = { extractVariable(L) for C in Clauses for L in C } - Forbidden
    return arb(Variables)

Given a set of clauses $\texttt{Cs}$ and a literal $l$, the procedure $\texttt{reduce}(C, l)$ performs all unit cuts and all unit subsumptions on clauses of $C$ that are possible using the unit clause $\{l\}$.  The resulting set of clauses is returned.  Mathematically, the function $\texttt{reduce}$ satifies the following specification:
$$\texttt{reduce}(\texttt{Cs},l)  = 
 \Bigl\{\, C \backslash \bigl\{\overline{\,l\,}\bigr\} \;|\; C \in \texttt{Cs} \wedge \overline{\,l\,} \in C \,\Bigr\} 
       \,\cup\, \Bigl\{\, C \in \texttt{Cs} \mid \overline{\,l\,} \not\in C \wedge l \not\in C \Bigr\} \cup \bigl\{\{l\}\bigr\}.
$$


In [6]:
def reduce(Clauses, l):
    lBar = complement(l)
    return   { C - { lBar } for C in Clauses if lBar in C }          \
           | { C for C in Clauses if lBar not in C and l not in C }  \
           | { frozenset({l}) }

$\texttt{Clauses}$ is a set of clauses.  The call $\texttt{saturate}(\texttt{Clauses})$ computes the set of those clauses that can be derived from $S$ via unit cuts or unit subsumptions.

In [7]:
def saturate(Clauses):
    S     = Clauses.copy()
    Units = { C for C in S if len(C) == 1 }
    Used  = set()                           # remember which unit clauses have already been used
    while len(Units) > 0:
        unit  = Units.pop()
        Used |= { unit }
        l     = arb(unit)
        S     = reduce(S, l)
        Units = { C for C in S if len(C) == 1 } - Used        
    return S

The function $\texttt{solve}(\texttt{Clauses}, \texttt{Literals})$ takes a set of clauses and a set of Literals as input.  The function tries to compute a variable assignment that makes all of the clauses true.  If this is successful, a set of unit clauses is returned.  This set of unit clauses does not contain  any complementary literals and therefore corresponds to a variable assignment satisfying all clauses.  If clauses is unsatisfiable, the set containing the  empty clause is returned.

The argument $\texttt{Literals}$ is a set containing all those literals that have already been used to reduce the clauses.  Initially, this set is empty.

In [ ]:
def solve(Clauses, Variables):
    S      = saturate(Clauses);
    empty  = frozenset()
    Falsum = {empty}
    if empty in S:
        return Falsum
    if all(len(C) == 1 for C in S):
        return S
    p      = selectVariable(S, Variables)
    negP   = complement(p)
    Result = solve(S | { frozenset({p}) }, Variables | { p })
    if Result != Falsum:
        return Result
    return solve(S | { frozenset({negP}) }, Variables| { p })

The function $\texttt{toString}(S)$ takes a set $S$ as input.  The set $S$ is a set of frozensets and the function converts $S$ into a string that looks like a set of sets. 

In [ ]:
def toString(S):
    "Convert the set S of frozen sets to a string where frozen sets are written as sets."
    if S == set():
        return '{}'
    result = '{ '
    for f in S:
        result += str(set(f)) + ', '
    result = result[:-2]
    result += ' }'
    return result
    

In [ ]:
c1 = frozenset({ 'r', 'p', 's' })
c2 = frozenset({ 'r', 's' })
c3 = frozenset({ 'p', 'q', 's' })
c4 = frozenset({ ('¬', 'p'), ('¬', 'q') })
c5 = frozenset({ ('¬', 'p'), 's', ('¬', 'r') })
c6 = frozenset({ 'p', ('¬', 'q'), 'r'})
c7 = frozenset({ ('¬', 'r'), ('¬', 's'), 'q' })
c8 = frozenset({ ('¬', 'p'), ('¬', 's')})
c9 = frozenset({ 'p', ('¬', 'r'), ('¬', 'q') })
c0 = frozenset({ ('¬', 'p'), 'r', 'q', ('¬', 's') })
S  = { c0, c1, c2, c3, c4, c5, c6, c7, c8, c9 }
toString(solve(S, set()))